# rascunho

In [ ]:
catalogs = prepare_catalogs(cluster_member_data, truth_data, cat1_columns, cat2_columns)

cat_clusters = catalogs[0]
cat_halos = catalogs[1]

In [ ]:
# match_info = matched_catalog_1[1][['cat1_id', 'cat2_id',  'is_central', 'richness', 'halo_mass', 'nmem_cat1', 'nmem_cat2', 'shared_num', 'shared_frac_cat1', 'shared_frac_cat2','cross_frac']].copy().drop_duplicates(subset=['cat1_id', 'cat2_id'])
# match_info

### Cross 

In [ ]:
match_tuples = list(match_info[['cat1_id', 'cat2_id', 'cross_frac']].itertuples(index=False, name=None))
# match_tuples

In [ ]:
# pairs = [(1,1,0.6), (2,1,0.4), (2,2,0.2),(3,2,0.01), (2,3,0.5)]

pairs = match_tuples

G = nx.Graph()
# add edges with 'weight' attribute
for a,b,w in pairs:
    G.add_edge(("A",a), ("B",b), weight=w)

random.seed(42)

# --- parameters --------------------------------------------------------------
n_components = 5
nodes_per_side = 10     # per component
edge_prob = 0.3         # sparsity inside each component
# ------------------------------------------------------------------------------

total_weight = 0.0
global_matching = set()

for i, nodes in enumerate(nx.connected_components(G)):
    subG = G.subgraph(nodes)
    m = nx.max_weight_matching(subG, maxcardinality=False)
    w = sum(subG[u][v]['weight'] for u, v in m)
    print("\nNodes in component:")
    for a in subG:
        if a[0] == 'A':
            for b in subG[a]:
                if b[0] == 'B':
                    print(a, b, subG[a][b]["weight"])
    total_weight += w
    global_matching |= m
    print(f"Component {i}: {len(subG)} nodes, {len(m)} pairs, weight = {w:.3f}")
    for u, v in m:
        if u[0] == "A":
            a, b = u, v
        else:
            a, b = v, u
        print(a, b)


print(f"\nTotal matching weight = {total_weight:.3f}")
print(f"Total pairs matched   = {len(global_matching)}")



In [ ]:
# global_matching

rows = []
for row_data in global_matching:
    row_dict = dict(row_data) 
    rows.append(row_dict)

# Cria o DataFrame
df = pd.DataFrame(rows)

In [ ]:
df

In [ ]:
df_ab = df.rename(columns={
    'A': 'cat1_id',
    'B': 'cat2_id'
})

In [ ]:
match = pd.merge(matched_catalog.drop_duplicates(subset=['cat1_id', 'cat2_id']), df_ab, how='inner', on=['cat1_id', 'cat2_id'])

In [ ]:
# m.columns

match

In [ ]:
# diff = np.isin(match['cat1_id'], matched_catalog_1[1]['cat1_id'], invert=True)

# match[diff]

In [ ]:
plt.scatter(np.log(m['halo_mass']), np.log(m['richness']))

In [ ]:
RM_cat_bij[RM_cat_bij['id'] == '1597']

In [ ]:
match_info[match_info['cat1_id'] == 1597]

In [ ]:
match_info[match_info['cat2_id'] == 1570200057279]

### functions

# Rascunho

In [26]:
import math

import sys

import os
import numpy as np
import pandas as pd

from astropy.io import fits
from astropy.table import Table

import matplotlib
from matplotlib import pyplot as plt
# %matplotlib inline

sys.path.insert(0, "/global/homes/c/cinlima/gcrcatalogs-new/gcr-catalogs")
sys.path.insert(0, "global/homes/c/cinlima/clevar/clevar")

import GCRCatalogs
#Put nerc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")

from numcosmo_py import Ncm, Nc
from numcosmo_py.external.pyssc import pyssc as PySSC
Ncm.cfg_init()

# from clevar import ClCatalog, MemCatalog
# from clevar.match import MembershipMatch, get_matched_pairs

# from clevar.match import output_matched_catalog
# import clevar
# clevar.__version__

sys.path.insert(0, 'NumCosmo/notebooks/richness_proxy/CatalogsMatching/')

# matplotlib.rcParams.update({'font.size': 22})
# matplotlib.rcParams.update({'figure.figsize': (10,8)})



In [27]:
# gc = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')


In [28]:
# mdata = Table.read('CatalogsMatching/test.fits')
# data = mdata[len(mdata['mass']) > 0]
# mdata

test_matchdata = Table.read('CatalogsMatching/match_ID.fits')

rich_data = test_matchdata["richness"]
z_data = test_matchdata["redshift"]
mass_data = test_matchdata["halo_mass"]

data = Table([mass_data, rich_data, z_data], names=('mass', 'richness', 'redshift'))
# rich_data

In [29]:
#Data threshold
logMmin = 13.0
logMmax = 15.0
zmin    = 0.0
zmax    = 1.2

area = 439.78987 

cosmo = Nc.HICosmoDEXcdm()
cosmo.omega_x2omega_k()
cosmo.param_set_by_name("H0", 67.74)
cosmo.param_set_by_name("Omegab", 0.0486)
cosmo.param_set_by_name("Omegac",0.3089 - 0.0486) #0.2603
cosmo.param_set_by_name("Omegak", 0.00)
cosmo.param_set_by_name("w", -1.0) #-1.0

prim = Nc.HIPrimPowerLaw.new()
prim.param_set_by_name("ln10e10ASA",3.0116707481421923)
prim.param_set_by_name("n_SA", 0.9667)

cosmo.add_submodel(prim)

dist = Nc.Distance.new(2.0)

tf = Nc.TransferFuncEH()

psml = Nc.PowspecMLTransfer.new(tf)
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()

# cosmo.set_property("Omegac_fit", True)
# cosmo.set_property("w_fit", True)
# prim.set_property("ln10e10ASA_fit", False)

#mass proxy
# cluster_m = Nc.ClusterMassNodist(lnM_min=np.log(10) * (logMmin - np.log10(0.6774)), lnM_max=np.log(10.0) * (logMmax - np.log10(0.6774)))
# cluster_m = Nc.ClusterMassLnrichExt(use_ln1pz = True)
cluster_m = Nc.ClusterMassAscaso() #lnRichness_min=np.log(rich_data.min()), lnRichness_max=6


# cmparam = [("mu", 4.301849), ("muM1", 1.463218), ("muZ1", 0.702439), 
#            ("muM2", 0.226122), ("muZ2", -1.029253), ("muMZ", 0.282406), 
#            ("sigma0", 0.255932), ("sigmaM1", -0.466271), ("sigmaZ1", -0.298706),
#            ("sigmaM2", -0.058500), ("sigmaZ2", 0.1567059), ("sigmaMZ", -0.0998257 )]

# for p, v in cmparam:
#     cluster_m.param_set_by_name(p, v)

cluster_m.param_set_by_name("cut", np.log(rich_data.min()))

cluster_m.param_set_by_name("mup0", 4.13094043037743)
cluster_m.param_set_by_name("mup1", 1.13300135028043)
cluster_m.param_set_by_name("mup2", 0.306957319158102)
cluster_m.param_set_by_name("sigmap0", 0.458308274798465)
cluster_m.param_set_by_name("sigmap1", -0.072016207741209)
cluster_m.param_set_by_name("sigmap2", -0.0777824945901317)

#photo-z
cluster_z = Nc.ClusterRedshiftNodist (z_min = zmin, z_max = zmax)

hmf_Jenkins = Nc.MultiplicityFuncJenkins.new()

mulf = Nc.MultiplicityFuncTinker.new()
mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)

hmf = Nc.HaloMassFunction.new(dist, psf, hmf_Jenkins)    
hmf.prepare(cosmo)
hmf.set_area(area * (np.pi / 180) ** 2)

cad = Nc.ClusterAbundance.new(hmf, None)
cad.prepare(cosmo , cluster_z , cluster_m)

mset = Ncm.MSet.new_array([cosmo, cluster_m, cluster_z])
ncdata = Nc.DataClusterNCount.new(cad, "NcClusterRedshiftNodist", "NcClusterMassAscaso")
ncdata.set_init(True)

lnR = np.log(np.array(rich_data))
redshift = np.array(z_data)

ncdata.set_lnM_obs(Ncm.Matrix.new_array(lnR, 1))
ncdata.set_z_obs(Ncm.Matrix.new_array(redshift, 1))
ncdata.set_lnM_true(Ncm.Vector.new_array(lnR))
ncdata.set_z_true(Ncm.Vector.new_array(redshift))
               
dset = Ncm.Dataset.new_array([ncdata])
lh = Ncm.Likelihood.new(dset)

In [30]:
mset["NcHICosmo"].param_set_desc("Omegac", {"fit": True})
mset["NcHICosmo"].param_set_desc("w", {"fit": True})
mset["NcHICosmo"].param_set_desc("Omegab", {"fit": True})

# mset.param_set_all_ftype(Ncm.ParamType.FREE)
mset.prepare_fparam_map()
mset.pretty_log()

#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  67.74               [FIXED]
#   -     Omegac[01]:  0.2603              [FREE]
#   -     Omegak[02]:  0                   [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0486              [FREE]
#   -          w[07]: -1                   [FREE]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law model for primordial spectra
#----------------------------------------------------------------------------------
# Model parameters
#   - ln10e10ASA[00]:  3.01167074814219    [FIXED]
#   - T_SA_ratio[01]:  0.2                 [

In [31]:
fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL )

In [32]:
fit.m2lnL_val()

-49271.673103925976

In [ ]:
fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)

#----------------------------------------------------------------------------------
# Model fitting. Interating using:
#  - solver:            NLOpt:ln-neldermead
#  - differentiation:   Numerical differentiantion (central)
...................
#  Minimum found with precision: |df|/f =  1.00000e-08 and |dx| =  1.00000e-05
#  Elapsed time: 00 days, 00:00:40.0830930
#  iteration            [000106]
#  function evaluations [000108]
#  gradient evaluations [000000]
#  degrees of freedom   [039597]
#  m2lnL     =    -66733.0634959476 (    -66733.063 )
#  Fit parameters:
#     0.483472665144004      0.03                   0                     
#----------------------------------------------------------------------------------
# Restarting:              yes
#  - absolute improvement: 17461.3903920217      
#  - relative improvement: -0.354390043853216    
#  - m2lnL_0  :            -49271.673103926      
#  - m2lnL_1  :            -66733.0634959476     
#--------------------------------------

In [ ]:
fit.log_info()

In [ ]:
N_WALKERS = 1200
N_RUN = 500
  
Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(N_WALKERS, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, N_WALKERS, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file('test1_mcmc')

esmcmc.start_run()
esmcmc.run(N_RUN)  
esmcmc.end_run()

esmcmc.mean_covar()

In [ ]:
# experiment = Ncm.ObjDictStr()
# experiment.set("likelihood", likelihood)
# experiment.set("model-set", mset)

In [ ]:
# ser = Ncm.Serialize.new(Ncm.SerializeOpt.CLEAN_DUP)
# ser.dict_str_to_yaml_file(experiment, "test2.yaml")
# ser.array_to_yaml_file(mfunc_oa, "test2.functions.yaml")

In [ ]:
## Halo catalog m200c --------------------------------------------------------------------------------------------
# DC2_halos_m200c = fits.open(
#     "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits"
# )
# dt_halos = Table(DC2_halos_m200c[1].data)
# # dt_halos['redshift_true', 'mass_fof', 'm200c', 'richness']



## Clevar catalog ------------------------------------------------------------------------------------------------

# min_richness = 5
# min_halo_mass = 1e13 #Msun

# #Halos/Clusters
# #SkySim + RedMaPPer: 'skysim5000_v1.1.1_redmapper_v0.8.5'
# RM_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

# #SkySim: 'skysim5000_v1.1.1_image'
# halo_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

# #Members
# RM_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
# halo_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

# #Match catalogs
# mt = MembershipMatch()

# mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/SS_RM_match_cross_membership')

# RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

# clusters_dt = Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']], names=('m200c', 'richness', 'redshift_true'))

# dt_halos = clusters_dt

In [ ]:
# nwalkers = 1000
# burnin = 40

# ntests = 100

# mcat = Ncm.MSetCatalog.new_from_file_ro("richness_cut/asc_rmin_"+str(rich_cut)+".fits", nwalkers * burnin)

# mcat.log_current_chain_stats()
# mcat.calc_max_ess_time(ntests, Ncm.FitRunMsgs.SIMPLE)
# mcat.calc_heidel_diag(ntests, 0.0, Ncm.FitRunMsgs.SIMPLE)

# # mset.pretty_log()
# # mcat.log_full_covar()
# mcat.log_current_stats()

# be2, post_lnnorm_sd = mcat.get_post_lnnorm()
# lnevol, glnvol = mcat.get_post_lnvol(0.6827)

# Ncm.cfg_msg_sepa()
# print(
#     "# Bayesian evidence:                                 % 22.15g +/- % 22.15g"
#     % (be2, post_lnnorm_sd)
# )
# print("# 1 sigma posterior volume:                          % 22.15g" % lnevol)
# print("# 1 sigma posterior volume (Gaussian approximation): % 22.15g" % glnvol)

In [ ]:
# min_richness = 5
# min_halo_mass = 1e13 #Msun

# #Halos/Clusters
# #CosmoDC2 + RedMaPPer: 'cosmoDC2_v1.1.4_redmapper_v0.8.1'
# RM_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

# #CosmoDC2: 'cosmoDC2_v1.1.4_image'
# halo_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

# #Members
# RM_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
# halo_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

# #Match catalogs
# mt = MembershipMatch()

# mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/cosmoDC2_RM_match_cross_membership')

# #Matched Data
# RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

# cdc2_dt = Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']], names=('m200c', 'richness', 'redshift_true'))
# dt_halos = cdc2_dt


# fit = catalog_fit(dt_halos, ascaso=False)


In [ ]:
# nwalkers = 1500

# Ncm.func_eval_set_max_threads(2)
# Ncm.func_eval_log_pool_stats()

# init_sampler = Ncm.MSetTransKernGauss.new(0)
# init_sampler.set_mset(mset)
# init_sampler.set_prior_from_mset()
# init_sampler.set_cov_from_rescale(1.0)

# apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

# esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
# esmcmc.set_nthreads(2)
# esmcmc.set_data_file("mcmc2.fits")

# esmcmc.start_run()
# esmcmc.run(1000)  
# esmcmc.end_run()

# esmcmc.mean_covar()


In [ ]:
# fit.log_covar()

In [ ]:
# data_fit_full = pd.DataFrame(fits.open("mcmc2.fits")[1].data).iloc[:, 1:14].T
# data_fit_void = np.array(data_fit_full)
# data_fit = []
# for item in data_fit_void:
#     arr= np.array(item)
#     data_fit.append(np.asarray(arr.tolist()))

# names = [
#     '1',
#     '2',
#     '3',
#     '4',
#     '5',
#     '6',
#     '7',
#     '8',
#     '9',
#     '10',
#     '11',
#     '12',
#     '13'
# ]
# labels=["\mu_0", "\mu_{M1}", "\mu_{M2}", "\mu_{Z1}"," \mu_{Z2}", "\mu_{MZ}", "\sigma_0", "\sigma_{M1}", "\sigma_{M2}", "\sigma_{Z1}", "\sigma_{Z2}", "\sigma_{MZ}","A_0" ]
# settings = {
#     "mult_bias_correction_order": 0,
#     "smooth_scale_2D": 3,
#     "smooth_scale_1D": 3,
#     "boundary_correction_order": 0,
# }
# samples3 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
# samples3.removeBurn(0.3)

In [ ]:
# # Triangle plot
# g2 = plots.get_subplot_plotter()

# g2.settings.lab_fontsize = 40
# g2.settings.legend_fontsize = 40
# g2.settings.axes_fontsize = 30

# g2.triangle_plot(
#     [samples3],
#     filled=True,
#     contour_ls="-",
#     contour_lws=1,
#     fine_bins=1,
#     colors=['#b186f1'],
#     fontsize=70,
#     line_args=[{'lw':1.2,'ls':'-', 'color':'#b186f1'}],
#     legend_labels=["Estendido", ],
# )
# plt.savefig('mcmc2.png')
# plt.show()


In [ ]:
# for i in range(1, 14):
#     display(Math(samples3.getInlineLatex(str(i),limit=1)))


In [ ]:
# Mset Catalog functions

# N_WALKERS = 1200
# N_RUN = 100
# BURNIN = 50

# MODEL = 'ext'
# FILE_NAME1 = "mass_richness_cut/"+MODEL+"_rmin_"+str(20)+"_mmin_"+str(1e14)+".fits"    
# mcat1 = Ncm.MSetCatalog.new_from_file_ro(FILE_NAME1, N_WALKERS * BURNIN)
# mset1 = mcat1.get_mset()
# muM2 = mset1.fparam_get(2)

# bla = mcat1.peek_pstats()
# print(bla.get_sd(3))
# bla.peek_mean().log_vals('blabla', '%.2f', )

# mcat1.log_full_covar()  
# pi = mcat1.fparam_get_pi_by_name("muM2")
# print(pi)
# mum2 = mset.param_get(pi.mid, pi.pid)

# print(mcat1.peek_current_e_var().dup_array())

# print(mcat1.peek_pstats().get_sd(4))

# mcmc_sd = [mcat1.peek_pstats().get_sd(3) for in range (13) 
# mcat1_bestfit = mcat1.get_bestfit_row().dup_array()
# print(mcat1_bestfit)
# print(mcat1.get_mean().dup_array())
# print(mcat1_bestfit[4], mset1.fparam_get(3))
# mcat1.calc_ci_direct(0.9545)



In [ ]:
# rich_cut_list = [5,20,50]

# bef = pd.DataFrame(data = {'Min_Richness': [], 'BEQ': [], 'BEQ Err': [], 'BEL': [], 'BEL Err': [], 'BF': []})

# for rcut in rich_cut_list:
    
#     N_WALKERS = 1200
#     N_RUN = 100
#     BURNIN = 50
    
#     RICH_CUT = rcut
    
#     MODEL = 'ext'
#     FILE_NAME1 = "richness_cut/"+MODEL+"_rmin_"+str(RICH_CUT)+".fits"

#     MODEL2 = 'asc'
#     FILE_NAME2 = "richness_cut/"+MODEL2+"_rmin_"+str(RICH_CUT)+".fits"
    
#     mcat1 = Ncm.MSetCatalog.new_from_file_ro(FILE_NAME1, N_WALKERS * BURNIN)
#     be1, post_lnnorm_sd1 = mcat1.get_post_lnnorm()
#     # lnevol1, glnvol1 = mcat1.get_post_lnvol(0.6827)

#     mcat2 = Ncm.MSetCatalog.new_from_file_ro(FILE_NAME2, N_WALKERS * BURNIN)
#     be2, post_lnnorm_sd2 = mcat2.get_post_lnnorm()
#     # lnevol2, glnvol2 = mcat2.get_post_lnvol(0.6827)

#     bf = np.exp(be1 - be2)
#     bef = pd.concat([bef, pd.DataFrame([{'Min_Richness': rcut, 'BEQ': be1, 'BEQ Err': post_lnnorm_sd1, 'BEL': be2, 'BEL Err':post_lnnorm_sd2, 'BF': bf}])], ignore_index=True)


### Ext (z) Fit Params


Here we do the fitting of quadratic model parameters using training data in **data_train**. Dependence on $z$ is considered:

\begin{equation}\label{M_R_3}
\begin{split}
    \left<\ln R | M, z \right> = \mu_0 + 
    \mu_{M1} \ln\left( \frac{M}{M_0} \right) +
    \mu_{M2} (\ln\left(\frac{M}{M_0} \right))^2 +\\
    + \mu_{z1} \left(\frac{ z}{z_0}\right) +
    \mu_{z2} \left(\frac{z}{z_0}\right)^2 +
    \mu_{Mz} \left(\frac{z}{z_0}\right)\ln\left(\frac{M}{M_0} \right);
\end{split}
\end{equation}

$$\textbf{p}_{2} = \{\mu_0, \mu_{M1}, \mu_{M2}, \mu_{Z1}, \mu_{Z2}, \mu_{MZ}, \sigma_0, \sigma_{M1}, \sigma_{M2}, \sigma_{Z1}, \sigma_{Z2}, \sigma_{MZ} \}.$$

In [ ]:
# lnR_mean_ext_z, lnR_std_ext_z, ext_z, data_train, data_test = fm.model_fit('ext_z', training = True)

In [4]:
import pandas as pd

# Exemplo de DataFrame com colunas 'x' e 'y'
data = {'x': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'y': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}
df = pd.DataFrame(data)

# Número fixo de elementos por bin
n_elementos_por_bin = 5

# Para garantir que as divisões são feitas corretamente, podemos ordenar as colunas
df_sorted = df.sort_values(by=['x', 'y'])

# Função para criar bins
def create_bins(df, column, n_elements):
    # Calcular o número de bins necessários
    n_bins = len(df) // n_elements
    if len(df) % n_elements != 0:
        n_bins += 1
    
    # Dividir os dados em bins (com base no número de elementos)
    df['bin'] = pd.cut(df[column], bins=n_bins, labels=False)
    return df

# Subdividindo os dados com base na coluna 'x' e 'y'
df_binned_x = create_bins(df_sorted, 'x', n_elementos_por_bin)
df_binned_y = create_bins(df_sorted, 'y', n_elementos_por_bin)

# Separar em DataFrames distintos por cada bin de 'x' e 'y'
df_x_bins = [df_binned_x[df_binned_x['bin'] == i] for i in df_binned_x['bin'].unique()]
df_y_bins = [df_binned_y[df_binned_y['bin'] == i] for i in df_binned_y['bin'].unique()]

# Resultado
for i, bin_df in enumerate(df_x_bins, 1):
    print(f"Bin X {i}:\n", bin_df, "\n")
    
for i, bin_df in enumerate(df_y_bins, 1):
    print(f"Bin Y {i}:\n", bin_df, "\n")

Bin X 1:
    x   y  bin
0  1  11    0
1  2  12    0
2  3  13    0
3  4  14    0
4  5  15    0 

Bin X 2:
     x   y  bin
5   6  16    1
6   7  17    1
7   8  18    1
8   9  19    1
9  10  20    1 

Bin Y 1:
    x   y  bin
0  1  11    0
1  2  12    0
2  3  13    0
3  4  14    0
4  5  15    0 

Bin Y 2:
     x   y  bin
5   6  16    1
6   7  17    1
7   8  18    1
8   9  19    1
9  10  20    1 

